In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import time
import os
import copy


cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
num_epochs = 10
num_classes = 6
batch_size = 128
learning_rate = 0.001

In [5]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
       transforms.RandomResizedCrop(224),
       transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
 data_dir = '/content/drive/MyDrive/classification_dataset'


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),data_transforms[x]) for x in ['train']}

dataloaders =  {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train']}

class_names = image_datasets['train'].classes

print(dataset_sizes)
print(class_names)


image_testsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['test']}
testloaders = {x: torch.utils.data.DataLoader(image_testsets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['test']}

{'train': 543}
['bear', 'butterfly', 'camel', 'chimp', 'duck', 'elephant']


In [7]:
model = torchvision.models.resnet18(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [8]:
model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)

In [9]:


model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized 
optimizer_fc = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_fc, step_size=7, gamma=0.1)

In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):  

    total_step = len(dataloaders)
    for epoch in range(num_epochs):
      for i,(images, classes) in enumerate(dataloaders['train']):
        images = images.to(device)
        labels = classes.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad() #sets dL/dw =0 
        loss.backward()       # computes dL/dw
        optimizer.step()      # w = w - lr*dL/dw
        
        if (i+1) % 5 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [14]:
 train_model(model, criterion, optimizer_fc, exp_lr_scheduler, 25)

Epoch [1/25], Step [5/1], Loss: 0.3451
Epoch [2/25], Step [5/1], Loss: 0.3187
Epoch [3/25], Step [5/1], Loss: 0.3775
Epoch [4/25], Step [5/1], Loss: 0.1564
Epoch [5/25], Step [5/1], Loss: 0.3343
Epoch [6/25], Step [5/1], Loss: 0.2625
Epoch [7/25], Step [5/1], Loss: 0.3616
Epoch [8/25], Step [5/1], Loss: 0.3269
Epoch [9/25], Step [5/1], Loss: 0.2112
Epoch [10/25], Step [5/1], Loss: 0.2657
Epoch [11/25], Step [5/1], Loss: 0.3943
Epoch [12/25], Step [5/1], Loss: 0.2870
Epoch [13/25], Step [5/1], Loss: 0.2705
Epoch [14/25], Step [5/1], Loss: 0.3110
Epoch [15/25], Step [5/1], Loss: 0.3235
Epoch [16/25], Step [5/1], Loss: 0.2557
Epoch [17/25], Step [5/1], Loss: 0.4656
Epoch [18/25], Step [5/1], Loss: 0.1950
Epoch [19/25], Step [5/1], Loss: 0.4498
Epoch [20/25], Step [5/1], Loss: 0.2724
Epoch [21/25], Step [5/1], Loss: 0.2115
Epoch [22/25], Step [5/1], Loss: 0.1828
Epoch [23/25], Step [5/1], Loss: 0.2186
Epoch [24/25], Step [5/1], Loss: 0.3472
Epoch [25/25], Step [5/1], Loss: 0.1636


In [12]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
outputs =[]
with torch.no_grad():
    correct = 0
    total = 0
    for images, classes in testloaders['test']:
        images = images.to(device)
        labels = classes.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

predicted = predicted.cpu().numpy()
labels = labels.cpu().numpy()

from sklearn import metrics
print(metrics.classification_report(labels,predicted))


Test Accuracy of the model on the test images: 95.0 %
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        20
           1       1.00      1.00      1.00        20
           2       0.91      1.00      0.95        20
           3       0.87      1.00      0.93        20
           4       1.00      0.90      0.95        20
           5       1.00      0.90      0.95        20

    accuracy                           0.95       120
   macro avg       0.95      0.95      0.95       120
weighted avg       0.95      0.95      0.95       120

